In [ ]:
import os
import numpy as np
import pathlib
from pyometiff import OMETIFFReader
import imageio
from PIL import Image
import cv2

import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

##### In this notebook, we prepare H&E images for conversion in a Pix2pix GAN. We want to predict IF images from the H&E images so we can match them up with the Xenium output. 
This assumes that the Pix2pix GAN has already been trained

In [ ]:
### Path to the Xenium output and H&E staining for a corresponding experiment.
path_to_HE_ome = "/mnt/sata1/Analysis_Alex/human_r1/h_and_e/hSI_09_r2.ome.tif"
xenium_path = "/mnt/sata1/Analysis_Alex/human_r1/human_09_r2/xenium_output"

In [ ]:
img_array_HE_orig = imageio.imread_v2(path_to_HE_ome)
h_and_e = img_array_HE_orig.astype(np.uint16)

Chunk the images

In [ ]:
from tqdm.notebook import tqdm

image_chunks = []
for row in tqdm(range(256, np.shape(h_and_e)[0] + 256, 256)):
    for col in range(256, np.shape(h_and_e)[1] + 256, 256):
        right_part = h_and_e[row - 256 : row, col - 256 : col, :]
        left_part = h_and_e[row - 256 : row, col - 256 : col, :]
        image_chunks.append(np.hstack([left_part, right_part]))

### convert images with gan

To convert images with the Pix 2 pix gan, remove all the files from the pix2pix test directory. Next, run the following cell.

In [ ]:
for i, arr in enumerate(image_chunks):
    # Convert the numpy array to an image
    img = Image.fromarray(arr.astype("uint8"))

    # Save the image to a file
    img.save(
        os.path.join(
            "/home/amonell/piloting/pytorch-CycleGAN-and-pix2pix/datasets/histology/test",
            f"image_{i + 1}.jpg",
        )
    )

After running the above cell, go to the pix2pix directory, activate the pytorch conda environment and enter the command 


python test.py --dataroot ./datasets/histology --name histology_pix2pix --model pix2pix --direction BtoA
